# Merging Datasets
# 02_EDA_merging_files

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 22/09/25 | Adrienne | Update | Made significant changes to program | 
| 23/09/25 | Adrienne | Update | Restructured file to merge EOB and Claim |

### Content

* [Introduction](#introduction)

Through EDA we have discovered that the coverage and claim response files do not contain any information that would be useful for our unsupervised learning task.  At this time, we are doing more EDA into the Patient file.  This program will focus on merging the Claim and EOB files.

EOB does not have Patient Medicare Number and Claim does not have Patient Number, so the files will be merged on Unique claim ID.  The benefit of using the Patient file would be having both patient identifiers in it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines

In [2]:
# readin clean datafiles
path = "../data/clean/"
eob_df = pd.read_pickle(path + 'explanation_of_benefit_clean.pkl')
coverage_df = pd.read_pickle(path + 'coverage.pkl')
claim_df = pd.read_pickle(path + 'claim_sample.pkl')
#claim_df =  pd.read_pickle(path + 'claim.pkl')
#claim_response_df =  pd.read_pickle(path + 'claim_response.pkl')
patient_df = pd.read_pickle(path + 'patient.pkl')

In [17]:
claim_df = pd.read_pickle(path + 'claim_sample.pkl')

Creating some common variables across datafiles

In [3]:
# Preprocess identifiers in EOB
#eob_df['patient_medicare_number'] = eob_df['contained_0_identifier_0_value']
eob_df['patient_number'] = eob_df['id'].str.replace(r'[-inpatientout]', '', regex=True)
eob_df['unique_claim_ID'] = eob_df['claimId_2'].str.replace(r'[-]', '', regex=True)

In [89]:
eob_df.head(1000)

,careTeam,created,diagnosis,extension,id,insurance,insurer,item,meta,outcome,...,paymentAmount,paymentDate,providerId,totalChargeType,totalChargeCurrency,totalChargeAmount,claimType,eobType,patient_number,unique_claim_ID
0,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646806,[{'coverage': {'reference': 'Coverage/part-a--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:47:01.129+00:00...,complete,...,129.16,NaN,NaN,Claim Total Charge Amount,USD,129.16,institutional,INPATIENT,10000002646806,102852906
1,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646833,[{'coverage': {'reference': 'Coverage/part-a--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:47:01.129+00:00...,complete,...,134.47,NaN,NaN,Claim Total Charge Amount,USD,134.47,institutional,INPATIENT,10000002646833,102852934
2,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,NaN,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646839,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646839,102852942
3,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,NaN,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646843,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646843,102852947
4,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646848,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646848,102852953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:13+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002283177,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'category': {'coding': [{'...,{'lastUpdated': '2021-08-17T17:46:27.529+00:00...,complete,...,1458.04,NaN,UNKNOWN,Claim Total Charge Amount,USD,0.00,professional,CARRIER,10000002283177,102462390
996,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:13+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002283179,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'category': {'coding': [{'...,{'lastUpdated': '2021-08-17T17:46:27.529+00:00...,complete,...,1134.55,NaN,UNKNOWN,Claim Total Charge Amount,USD,0.00,professional,CARRIER,10000002283179,102462393
997,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:13+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002283181,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'category': {'coding': [{'...,{'lastUpdated': '2021-08-17T17:46:27.529+00:00...,complete,...,1128.27,NaN,UNKNOWN,Claim Total Charge Amount,USD,0.00,professional,CARRIER,10000002283181,102462396
998,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:13+00:00,[{'dia

In [77]:
# For ease of merging initially limiting dataset columns
eob_df_part = eob_df[['patient_number','unique_claim_ID' ]]
patient_df_part = patient_df[['patient_number', 'patient_medicare_number']]
claim_df_part = claim_df[['patient_medicare_number', 'unique_claim_ID']]
#claim_response_df_part = claim_response_df[['patient_medicare_number', 'unique_claim_ID']]


In [7]:
print(f"num of unique patient numbers in EOB: {len(eob_df['patient_number'].unique())}")
print(f"num of unique claim IDs in EOB: {len(eob_df['unique_claim_ID'].unique())}")
print(f"num of unique patient numbers in Patient: {len(patient_df['patient_number'].unique())}")
print(f"num of unique patient medicare numbers in Patient: {len(patient_df['patient_medicare_number'].unique())}")
print(f"num of unique patient medicare numbers in Claim: {len(claim_df['patient_medicare_number'].unique())}")
print(f"num of unique claim IDs in Claim: {len(claim_df['unique_claim_ID'].unique())}")
#print(f"num of unique patient medicare numbers in Claim Response: {len(claim_response_df['patient_medicare_number'].unique())}")

num of unique patient numbers in EOB: 4091
num of unique claim IDs in EOB: 4091
num of unique patient numbers in Patient: 5000
num of unique patient medicare numbers in Patient: 5000
num of unique patient medicare numbers in Claim: 2695
num of unique claim IDs in Claim: 19905


In [ ]:
claim_df['unique_claim_ID'] = claim_df['identifier_0_value'].str.replace(r'[-]', '', regex=True).str.replace('dcn', '')

In [71]:
# Checking lengths of values in columns
print(eob_df['patient_number'].str.len().unique())
print(eob_df['unique_claim_ID'].str.len().unique())
print(patient_df['patient_number'].str.len().unique())
print(patient_df['patient_medicare_number'].str.len().unique())
print(claim_df['patient_medicare_number'].str.len().unique())
print(claim_df['unique_claim_ID'].str.len().unique())
print(claim_df['identifier_0_value'].str.len().unique())
#print(claim_response_df['patient_medicare_number'].str.len().unique())

[14 11]
[9]
[14]
[11]
[11]
[ 9 19 14]
[10 13 23 15]


In [75]:
claim_df['length_counts'] = claim_df['unique_claim_ID'].str.len()
#claim_df['length_counts'] = claim_df['identifier_0_value'].str.len()
length_distribution = claim_df['length_counts'].value_counts()
length_distribution

length_counts
9     18564
19      861
14      575
Name: count, dtype: int64

In [ ]:
test_df = claim_df.head(1000)
#for i, rec in  enumerate(test_df.identifier_0_value):
for i, rec in  enumerate(test_df.unique_claim_ID):   
    if len(rec) != 9:
        print(f" rec {i} {rec}")

 rec 166 04e54d88f6f07a4a69e
 rec 205 1f87dcbf5f334978342
 rec 206 28a203d27baf47f3900
 rec 207 362e7664ba36443426f
 rec 208 77c2a1f38ec84373417
 rec 209 b3a8cc729eb3d6cef92
 rec 210 b3c9d7abbfe427e6cd5
 rec 211 c07fca0158246692b42
 rec 212 c09c14bb971aab54c26
 rec 213 d532ed09fbd11644886
 rec 214 d8f4e3d35cce950ba1b
 rec 215 e6a94ba3ed3ade74c79
 rec 216 fc67035d7b3ed63c1d7
 rec 217 0393985e735038
 rec 218 05ca0f6d266ddf
 rec 219 0c4deb07c99ad0
 rec 250 163edd77dd304d
 rec 251 27955e4ca0f87d
 rec 252 3a0d7bf8a7bf31
 rec 253 45ce4fc8d06a75
 rec 254 48925c395f3c7e
 rec 255 57761ebcadf51e
 rec 256 88e873bcd57695
 rec 257 97fa88fc8d333c
 rec 258 9830342a348c81
 rec 259 bd04e4f84eaf90
 rec 260 bf0fb424a78f01
 rec 261 c511d221e85c5b
 rec 262 dbf16a3c865ca6
 rec 263 dcdb45d9367d01
 rec 696 0124f1889fa3b513059
 rec 697 015752206685bb93aec
 rec 698 02eb9f39f7bded2170b
 rec 699 0b21e4473099e17d5e4
 rec 700 0cf0f6d964ed7feeec0
 rec 701 0dadf5db840fcf13e1f
 rec 702 0e528052f86fa4c2c7f
 rec 703 187

In [88]:
claim_df.tail(100)

,billablePeriod_end,billablePeriod_start,contained_0_birthDate,contained_0_extension_0_url,contained_0_extension_0_valueCode,contained_0_gender,contained_0_id,contained_0_identifier_0_system,contained_0_identifier_0_type_coding_0_code,contained_0_identifier_0_type_coding_0_display,...,diagnosis_13_type_1_coding_0_code,diagnosis_13_type_1_coding_0_display,diagnosis_13_type_1_coding_0_system,patient_medicare_number,patient_number,patient_first_name,patient_last_name,unique_claim_ID,hcpcs_code,length_counts
19900,2021-06-22,2021-06-22,1948-12-28,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00KN25,#patient,Ignacio928,Pizarro791,100177893,99221,9
19901,2012-02-27,2012-02-27,1948-12-28,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00KN25,#patient,Ignacio928,Pizarro791,100177894,99241,9
19902,2016-02-26,2016-02-26,1948-12-28,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00KN25,#patient,Ignacio928,Pizarro791,100177899,99241,9
19903,2017-02-25,2017-02-25,1948-12-28,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00KN25,#patient,Ignacio928,Pizarro791,100177900,99241,9
19904,2017-12-29,2017-12-29,1948-12-28,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00KN25,#patient,Ignacio928,Pizarro791,100177901,99241,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2021-05-04,2021-05-04,NaN,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00HP87,#patient,G.,Robled,100169264,G0444,9
19996,2013-11-14,2013-11-14,1944-01-06,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00MY43,#patient,Alfonzo975,Marvin195,100250567,99241,9
19997,2015-03-19,2015-03-19,1944-01-06,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00MY43,#patient,Alfonzo975,Marvin195,100250568,G0444,9
19998,2016-06-09,2016-06-09,1944-01-06,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,male,patient,http://hl7.org/fhir/sid/us-mbi,MC,Patient's Medicare Number,...,NaN,NaN,NaN,1S00E00MY43,#patient,Alfonzo975,Marvin195,100250569,G0444,9


In [90]:
l1 = set(eob_df['patient_number'].unique()).intersection(patient_df['patient_number'])
len(l1)

0

Merging files together to see if most records are found in each file

In [78]:
df = pd.merge(eob_df_part, claim_df_part, how = 'outer', on = 'unique_claim_ID')

In [80]:
len(df)

24068

In [79]:
df.describe()

,patient_number,unique_claim_ID,patient_medicare_number
count,4114,24068,20000
unique,4091,17349,306
top,10000000014846,100123269,1S00E00KH30
freq,2,3,2142


In [40]:
df.patient_medicare_number.isnull().sum()

np.int64(4091)

In [82]:
df.head(100)

,patient_number,unique_claim_ID,patient_medicare_number
0,NaN,0026d6211ab0fe4b382,1S00E00AH76
1,NaN,00a503839064fe20620,1S00E00AH76
2,NaN,00cd9359bbf965,1S00E00AC99
3,NaN,00d28c81e9d6c53c750,1S00E00AH76
4,NaN,00fd0a8fa06c4c78671,1S00E00AH76
...,...,...,...
95,NaN,100000409,1S00E00GA15
96,NaN,100000410,1S00E00GA15
97,NaN,100000410,1S00E00GA15
98,NaN,100000411,1S00E00GA15


This following contains code for working with the Patient file that will be moved here for the time being

In [ ]:
# Preprocess Identifier in Patient
patient_df['patient_medicare_number'] = patient_df['identifier_1_value']
patient_df['patient_number'] = patient_df['identifier_0_value'].str.replace(r'[-]', '', regex=True)
patient_df['patient_first_name'] = patient_df['name_0_given'].str.replace(r'[ \[ \]"]', '', regex=True)
patient_df['patient_last_name'] = patient_df['name_0_family']

In [91]:
patient_df.head()

,address_0_state,birthDate,deceasedBoolean,extension_0_url,extension_0_valueCode,extension_1_url,extension_1_valueCoding_code,extension_1_valueCoding_display,extension_1_valueCoding_system,extension_2_extension_0_url,...,identifier_2_type_coding_0_extension_0_url,identifier_2_type_coding_0_extension_0_valueCoding_code,identifier_2_type_coding_0_extension_0_valueCoding_display,identifier_2_type_coding_0_extension_0_valueCoding_system,identifier_2_type_coding_0_system,identifier_2_value,patient_medicare_number,patient_number,patient_first_name,patient_last_name
0,22,1953-10-12,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248152002,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00FR92,10000000009392,Mina319,Mohr916
1,22,1946-03-01,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248152002,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00AK52,10000000000852,Leda374,Kshlerin58
2,22,1947-08-31,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,2,Black,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00FU79,10000000009579,Jeromy156,Morissette863
3,22,1952-01-13,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,5,Hispanic,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00CY77,10000000003977,Sergio619,Vanegas191
4,22,1943-10-05,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00AY95,10000000001995,Fernando603,Aufderhar910


In [ ]:
df = pd.merge(eob_df_part, patient_df_part, how = 'outer', on = 'patient_number')